# Fun graph problems

1. 3D Maze solver using
    - BFS
    - DFS

### Maze Generator

- The generated maze has dimensions (layers x rows x columns)
- The generated maze accepts the following inputs:
    - rows
    - columns
    - obstacles
    - layers (default 1)
- The generated maze produced will contain
    - 'S' for Start
    - 'E' for End
    - '#' for Obstacle
    - '.' for open path

In [142]:
import random
# utility functions for graph solvers

def print_maze(maze):
    layers, rows, columns = len(maze), len(maze[0]), len(maze[0][0])
    for layer in range(layers):
        print('\nfloor: ', layer)
        for row in range(rows):
            for col in range(columns):
                print(maze[layer][row][col], end = ' ')
            print()
            
def print_maze_path(maze, shortest_path):
    if not shortest_path:
        return 'No shortest path found'
    maze_path = maze.copy()
    for idx, path in enumerate(shortest_path):
        z, r, c = path
        maze_path[z][r][c] = str(idx)
    print_maze(maze_path)

def generate_maze(rows, columns, obstacles, layers = 1):
    def generate_flat_maze():
        samples = random.sample(range(size), obstacles + 2) # + 2 for the start and end
        flat_maze = ['.' for _ in range(size)]
        for index, sample in enumerate(samples):
            if index == 0:
                flat_maze[sample] = 'S'
            elif index == 1:
                flat_maze[sample] = 'E'
            else:
                flat_maze[sample] = '#'
        return flat_maze
    
    def reconstruct_maze(flat_maze):
        maze = [[[] for row in range(rows)] for layer in range(layers)]
        for layer in range(layers):
            for row in range(rows):
                for col in range(columns):
                    maze[layer][row].append(flat_maze.pop())
        return maze

    
    size = rows * columns * layers
    if obstacles + 2 >= size:
        raise Exception(f'Too many obstacles {obstacles} + start/end for the grid {size}')
    flat_maze = generate_flat_maze()
    maze = reconstruct_maze(flat_maze)
    print_maze(maze)
    return maze

def find_start(graph):
    ''' returns the starting location and checks there is an end'''
    start, end = False, False
    start_loc = None
    for layer in range(len(graph)):
        for row in range(len(graph[0])):
            for col in range(len(graph[0][0])):
                if graph[layer][row][col] == 'S':
                    start_loc = (layer, row, col)
                    start = True
                if graph[layer][row][col] == 'E':
                    end = True
    if not start:
        print('No Starting point found')
    if not end:
        print('No Ending point found')
    return start_loc

# Example usage of the generate_maze function

In [143]:
maze = generate_maze(rows = 5, columns = 10, obstacles = 50, layers = 3)


floor:  0
. # # . # # . . . # 
. . . . # # . . . # 
. # . . . # # . . . 
# # # # . # . . # # 
. . . . . # . . . . 

floor:  1
# . # . . . # # . . 
. . # . # . . . # . 
. # . . . . . . . . 
. # E # . # . # # . 
. . # . . # . # . . 

floor:  2
. # . # S . # # . # 
# . . . . . . . . . 
. . # . . . . # . . 
. # # . . # . . . . 
# # . . . . . # . # 


# BFS Solution

In [165]:
from collections import deque

def solve_maze_bfs(graph):
    def explore_neighbors(current_layer, current_row, current_col):
        '''Checks available paths to take,
        appends paths if constrains are satisfied'''
        layer_directions = [0, 0, 0, 0, 1, -1]
        row_directions = [-1, 1, 0, 0, 0, 0] 
        col_directions = [0, 0, 1, -1, 0, 0]
        
        for index in range(len(layer_directions)):
            updated_layer = current_layer + layer_directions[index]
            updated_row = current_row + row_directions[index]
            updated_col = current_col + col_directions[index]
            
            # boundary checks
            if updated_layer < 0 or updated_row < 0 or updated_col < 0:
                continue
            if updated_layer > len(graph) - 1 or updated_row > len(graph[0]) - 1 or updated_col > len(graph[0][0]) - 1:
                continue

            # constraint checks
            if graph[updated_layer][updated_row][updated_col] == '#':
                continue

            # final check to see if the path has already been explored
            if (updated_layer, updated_row, updated_col) in visited:
                continue
            else:
                visited.add((updated_layer, updated_row, updated_col))
                visited_path.append((updated_layer, updated_row, updated_col))
                path_queue.append(current_path + [(updated_layer, updated_row, updated_col)])

            # add to the search
            layer_queue.append(updated_layer)
            row_queue.append(updated_row)
            col_queue.append(updated_col)
            
    
    start_layer, start_row, start_col = find_start(graph)
    
    # separate queue for each dimension
    layer_queue = deque([start_layer])
    row_queue = deque([start_row])
    col_queue = deque([start_col])

    visited = set()
    visited.add((start_layer, start_row, start_col))
    visited_path = [(start_layer, start_row, start_col)]
    path_queue = deque([[(start_layer, start_row, start_col)]])
    
    while row_queue:
        current_layer = layer_queue.popleft()
        current_row = row_queue.popleft()
        current_col = col_queue.popleft()
        current_path = path_queue.popleft()
        if graph[current_layer][current_row][current_col] == 'E':
            return [visited_path, current_path]
        explore_neighbors(current_layer, current_row, current_col)
        
def main():
    maze = generate_maze(rows = 5, columns = 10, obstacles = 50, layers = 3)
    visited_path, solution_path = solve_maze_bfs(maze)
    s_solution_path = f'\noptimal solution solved in {len(solution_path)} steps'
    print(s_solution_path)
    print('*'*(len(s_solution_path)-1))
    print_maze_path(maze, solution_path)
    
    s_visited_path = f'\nbfs traversal solution solved in {len(visited_path)} iterations'
    print(s_visited_path)
    print('*'*(len(s_visited_path)-1))
    print_maze_path(maze, visited_path)

main()


floor:  0
# # # # # . # # . . 
# . . . . # . # . . 
# . . # # . . . . . 
. . . . . . # . . # 
. # # . # . . # . . 

floor:  1
. # . . E # # # . . 
. . . # . . . . . . 
# . # # . . . . # # 
. . . # . . . . # . 
# # . # # . . . # . 

floor:  2
. . # . . . . # . . 
# # S # . # . . . . 
. # . # . . . . . # 
. . # # # . . . # . 
. . . . . . . # . . 

optimal solution solved in 5 steps
**********************************

floor:  0
# # # # # . # # . . 
# . . . . # . # . . 
# . . # # . . . . . 
. . . . . . # . . # 
. # # . # . . # . . 

floor:  1
. # 2 3 4 # # # . . 
. . 1 # . . . . . . 
# . # # . . . . # # 
. . . # . . . . # . 
# # . # # . . . # . 

floor:  2
. . # . . . . # . . 
# # 0 # . # . . . . 
. # . # . . . . . # 
. . # # # . . . # . 
. . . . . . . # . . 

bfs traversal solution solved in 19 iterations
**********************************************

floor:  0
# # # # # . # # . . 
# 9 5 11 18 # . # . . 
# 15 10 # # . . . . . 
. . 17 . . . # . . # 
. # # . # . . # . . 

floor:  1
16 # 3

In [163]:
def solve_maze_dfs(graph):
    def explore_neighbors(current_layer, current_row, current_col, current_path):
        layer_directions = [0, 0, 0, 0, 1, -1]
        row_directions = [-1, 1, 0, 0, 0, 0]
        col_directions = [0, 0, 1, -1, 0, 0]
        for index in range(len(row_directions)):
            updated_layer = current_layer + layer_directions[index]
            updated_row = current_row + row_directions[index]
            updated_col = current_col + col_directions[index]

            # boundary checks
            if updated_row < 0 or updated_col < 0 or updated_layer < 0:
                continue
            if updated_row > len(graph[0]) - 1 or updated_col > len(graph[0][0]) - 1 or updated_layer > len(graph) - 1:
                continue

            # constraint checks
            if graph[updated_layer][updated_row][updated_col] == '#':
                continue

            # final check to see if the path has been explored already
            if (updated_layer, updated_row, updated_col) in visited:
                continue
            else:
                # NOTE TO SELF: You do not want to add visited here for dfs
                # If you do, you search a dfs with a bfs depth = 1 at each iteration
#                 visited.add((updated_layer, updated_row, updated_col))
#                 visited_path.append((updated_layer, updated_row, updated_col))
                stack.append((updated_layer, updated_row, updated_col))
                path.append(current_path + [(updated_layer, updated_row, updated_col)])

            
    start_layer, start_row, start_col = find_start(graph)
    visited = set()
    visited.add((start_layer, start_row, start_col))
    visited_path = [(start_layer, start_row, start_col)]
    stack = [(start_layer, start_row, start_col)]
    path = [[(start_layer, start_row, start_col)]]
    while stack:
        current_layer, current_row, current_col = stack.pop()
        current_path = path.pop()
        
        # NOTE TO SELF: Need to add visited here for dfs
        visited.add((current_layer, current_row, current_col))
        visited_path.append((current_layer, current_row, current_col))
        
        if graph[current_layer][current_row][current_col] == 'E':
            current_path.append((current_layer, current_row, current_col))
            return [visited_path, current_path]
        
        explore_neighbors(current_layer, current_row, current_col, current_path)

def main():
    maze = generate_maze(rows = 5, columns = 10, obstacles = 10, layers = 1)
    visited_path, solution_path = solve_maze_dfs(maze)
    s_solution_path = f'\noptimal solution solved in {len(solution_path)} steps'
    print(s_solution_path)
    print('*'*(len(s_solution_path)-1))
    print_maze_path(maze, solution_path)
    
    s_visited_path = f'\nbfs traversal solution solved in {len(visited_path)} iterations'
    print(s_visited_path)
    print('*'*(len(s_visited_path)-1))
    print_maze_path(maze, visited_path)
    
main()


floor:  0
. . # . . . # . . . 
. . # . . # . . . . 
# . # . . . . . . . 
. # . . # . . S . . 
# . . . # . E . . . 

optimal solution solved in 6 steps
**********************************

floor:  0
. . # . . . # . . . 
. . # . . # . . . . 
# . # . . . . . . . 
. # . . # 2 1 0 . . 
# . . . # 3 5 . . . 

bfs traversal solution solved in 6 iterations
*********************************************

floor:  0
. . # . . . # . . . 
. . # . . # . . . . 
# . # . . . . . . . 
. # . . # 3 2 1 . . 
# . . . # 4 5 . . . 
